# How many SSO alerts can we expect per night of the LSST?

This notebook presents the results of querying the mid point exposure times for all Solar System Object (SSO) diaSources from DP0.3 which is hosted on the Rubin Science Platform (RSP; https://data.lsst.cloud/). We determined the number of visits per night from the baseline_v3.0 survey simulation (which DP0.3 was built from). We use this to calculate the number of SSO alerts we expect LSST to generate per night and per visit.

We find that one should generally expect ~300 SSO alerts per LSST visit.

These numbers should be compared to the predicted number of alerts for all transients detected by LSST, as presented in DMTN-102 (https://dmtn-102.lsst.io/).

In [ ]:
%matplotlib notebook

In [ ]:
%matplotlib inline

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pandas as pd
import rubin_sim.maf as maf

# Calculate number of visits per night

In [ ]:
# select the survey simulation (DP0.3 was made using baseline_v3.0)
# Can be downloaded from: https://s3df.slac.stanford.edu/data/rubin/sim-data/sims_featureScheduler_runs3.0/baseline/
opsdb = "baseline_v3.0_10yrs.db"

In [ ]:
# load the visit database
data = maf.get_sim_data(
    opsdb, None, None, full_sql_query="select observationId,observationStartMJD from observations;"
)

# convert to dataframe
df = pd.DataFrame(data)

In [ ]:
df.sort_values("observationStartMJD")

In [ ]:
int(np.amin(df["observationStartMJD"])) - 0.5, int(np.amax(df["observationStartMJD"])) + 1.5

In [ ]:
# create nightly bins and count the number of visits per night
visit_nights = np.arange(
    int(np.amin(df["observationStartMJD"])) - 0.5, int(np.amax(df["observationStartMJD"])) + 1.5, 1
)
visit_counts, visit_bins = np.histogram(df["observationStartMJD"], bins=visit_nights)
visit_centres = (visit_nights[:-1] + visit_nights[1:]) / 2.0

In [ ]:
# plot of number of visits per night over the course of the 10yr survey
fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

ax1.scatter(visit_centres, visit_counts)

ax1.set_xlabel("night")
ax1.set_ylabel("visits per night")

plt.show()

# load the counted number of SSO alerts per night

This was calculated by querying DP0.3 on the RSP using a query like:

```
dia = service.search("SELECT dia.midPointMjdTai "
                        "FROM dp03_catalogs_10yr.DiaSource AS dia "
                        "WHERE dia.midPointMjdTai>={} AND dia.midPointMjdTai<{}".format(int(t0)-0.5,int(t1)+0.5)).to_table()
```

We split this query into 28 day chunks (querying the full midPointMjdTai seemed to crash the RSP notebook!). The results of each query was binned by night and the counts and night bins were saved in the alert_counts folder.

In [ ]:
# directory where the query results are saved
save_dir = "alert_counts"

# The queries where broken into 28 day chunks, using the following bins
mjd_min = np.amin(df["observationStartMJD"])
mjd_max = np.amax(df["observationStartMJD"])
mjd_nights = np.arange(int(mjd_min), int(mjd_max) + 28, 28)

# define empty lists to hold all results
all_nights = []
all_counts = []

# loop to load all query result files in order
for i in range(len(mjd_nights[1:])):

    t0 = mjd_nights[i]
    t1 = mjd_nights[i + 1]
    #     print(i,t0,t1)

    fname_count = "{}/counts_{}.npy".format(save_dir, i)
    fname_night = "{}/nights_{}.npy".format(save_dir, i)

    #     print("load: {} {}".format(fname_count,fname_night))
    counts = np.load(fname_count)
    nights = np.load(fname_night)

    # the righthand bins edges must be rejected to combine the separate chunks into one bin range spanning the full 10yrs
    all_nights += list(nights[:-2])
    all_counts += list(counts[:-1])


all_nights = np.append(np.array(all_nights), nights[-2])  # put the last righthand bin edge back
all_counts = np.array(all_counts)

# drop any nights over the survey duration
# (we used 28 day query ranges and now need 1 night results)
mask = all_nights <= visit_nights[-1]
all_nights = all_nights[mask]
all_counts = all_counts[mask[1:]]

# calculate bin centres
all_centres = (all_nights[:-1] + all_nights[1:]) / 2.0

# determine the mean number of alerts per visit
# this is the mean per night as we calculate total alerts per night/total visits per night
alerts_visit = all_counts / visit_counts

In [ ]:
all_nights

In [ ]:
all_counts

In [ ]:
len(all_counts), len(all_nights)

In [ ]:
len(visit_counts), len(visit_nights)

In [ ]:
visit_nights

In [ ]:
# plot of the total number of SSO alerts per night counted from DP0.3
fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

ax1.plot(all_centres, all_counts)

ax1.set_xlabel("night")
ax1.set_ylabel("SSO alerts per night")

fname = "SSO_alerts_per_night.pdf"
plt.savefig(fname, facecolor="w", transparent=True, bbox_inches="tight")

plt.show()

In [ ]:
# plot of the mean number of SSO alerts per visit (per night)
fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

ax1.plot(all_centres, alerts_visit)

med = np.nanmedian(alerts_visit)
ax1.axhline(med, c="r", label="median = {:.0f}".format(med))
mean = np.nanmean(alerts_visit)
ax1.axhline(mean, c="r", ls=":", label="mean = {:.0f}".format(mean))

ax1.set_xlabel("night")
ax1.set_ylabel("mean SSO alerts per visit (per night)")
ax1.legend()

fname = "SSO_alerts_per_visit.pdf"
# fname = "SSO_alerts_per_visit.png"
plt.savefig(fname, facecolor="w", transparent=True, bbox_inches="tight")

plt.show()

In [ ]:
# histogram of the mean number of SSO alerts per visit per night
fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

ax1.hist(alerts_visit, bins=50, histtype="step")

med = np.nanmedian(alerts_visit)
ax1.axvline(med, c="r", label="median = {:.0f}".format(med))
mean = np.nanmean(alerts_visit)
ax1.axvline(mean, c="r", ls=":", label="mean = {:.0f}".format(mean))

ax1.set_ylabel("number")
ax1.set_xlabel("mean SSO alerts per visit (per night)")
ax1.legend()

fname = "SSO_alerts_per_visit_hist.pdf"
# fname = "SSO_alerts_per_visit_hist.png"
plt.savefig(fname, facecolor="w", transparent=True, bbox_inches="tight")

plt.show()

From this analysis we can expect ~300 SSO alerts per visit for typical LSST visits (median value). Note that this number is an order of magnitude lower than the predicted ~3000 SSO alerts per visit in DMTN-102. In section 2.2 of that document it has been assumed that all main belt asteroids are evenly spread across the sky and they have assumed that all asteroids in the field of view of a visit would be detectable by LSST:

N_survey = 5.5e6 asteroids in LSST survey  
A_survey = 1.8e4 square degrees of LSST survey footprint  
A_visit = 9.6 square degrees footprint for a single visit

The estimated number of asteroids per visit is therefore:

N_visit = (N_survey/A_survey) * A_visit ~ 3000 asteroids per visit

This estimate does not take into account the effects of changing heliocentric/geocentric distance, or phase angle effects, on the brightness of an asteroid in the field of view of a visit. DP0.3 takes into account the distribution of small bodies across the sky and has calculated their ephemerides in order to obtain a predicted brightness at the time of each visit. Furthermore the predicted brightness for each asteroid has been calculated in different filters by using the surface colour which was assigned when the population was generated. We would therefore expect that only a fraction of asteroids in the field of view would be above the LSST detection limit (in that filter). Therefore the lower value of ~300 alerts per visit is a more accurate estimate of number of SSO alerts per visit during the survey.

In comparison to the number of other transients alerts in DP0.3, SSO alerts are not expected to dominate the total number of alerts:
- Variable Stars: ~7200 alerts per visit
- Supernovae: ~200 alerts per visit
- Active Galactic Nuclei: ~70 alerts per visit